In [2]:
import pandas
import numpy
import statsmodels.api as smf
import datetime

In [3]:
def normalize_weekday(weekday):
    """ returns 3 letter lower case weekday """
    short_upper_days = ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU']
    upper_days = ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN']
    lower_days = list(map(str.lower, upper_days))
    number_days = [0, 1, 2, 3, 4, 5, 6]
    if weekday in lower_days:
        return weekday
    if weekday in upper_days:
        return weekday.lower()
    if weekday in number_days:
        return lower_days[number_days.index(weekday)]
    if weekday.upper() in short_upper_days:
        return lower_days[short_upper_days.index(weekday.upper())]
    return ''

def add_shifted_columns(df, column):
    df[column + '_m1'] = df[column].shift(1)
    df[column + '_p1'] = df[column].shift(-1)
    df = df.fillna(0.0)
    return df

In [8]:
#df = pandas.read_csv('Breeo MMM Thursday Fun - data.csv', thousands=',')
df = pandas.read_csv('data_percent.csv')
df['wkday'] = df.date.apply(lambda x: normalize_weekday(datetime.datetime.strptime(x, '%Y-%m-%d').weekday()))
df = add_shifted_columns(df, 'bing_cost')
df = add_shifted_columns(df, 'pinterest_cost')
df = add_shifted_columns(df, 'google_ads_cost')
df = add_shifted_columns(df, 'hulu_cost')
df = add_shifted_columns(df, 'facebook_cost')
df = df[df.date > '2021-03-15']
df

,date,Revenue,bing_cost,pinterest_cost,google_ads_cost,hulu_cost,facebook_cost,total_spend,bing_percent,pinterest_percent,...,bing_cost_m1,bing_cost_p1,pinterest_cost_m1,pinterest_cost_p1,google_ads_cost_m1,google_ads_cost_p1,hulu_cost_m1,hulu_cost_p1,facebook_cost_m1,facebook_cost_p1
74,2021-03-16,118095.35,1375.41,2324.49,4945.77,0.00,10064.32,18709.99,0.07,0.12,...,1389.06,1165.41,2290.91,2315.41,5485.06,4568.04,0.00,0.00,9062.17,10390.00
75,2021-03-17,85401.42,1165.41,2315.41,4568.04,0.00,10390.00,18438.86,0.06,0.13,...,1375.41,1228.18,2324.49,2337.44,4945.77,5226.53,0.00,0.00,10064.32,10491.66
76,2021-03-18,94311.82,1228.18,2337.44,5226.53,0.00,10491.66,19283.81,0.06,0.12,...,1165.41,1232.38,2315.41,2333.17,4568.04,5788.53,0.00,0.00,10390.00,11286.71
77,2021-03-19,127975.43,1232.38,2333.17,5788.53,0.00,11286.71,20640.79,0.06,0.11,...,1228.18,1475.58,2337.44,2346.97,5226.53,10787.78,0.00,0.00,10491.66,12424.05
78,2021-03-20,136463.18,1475.58,2346.97,10787.78,0.00,12424.05,27034.38,0.05,0.09,...,1232.38,1713.82,2333.17,2343.99,5788.53,15358.29,0.00,0.00,11286.71,11158.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,2021-11-20,214066.61,1051.16,3000.00,28033.24,1068.66,17021.54,50174.60,0.02,0.06,...,896.01,1060.70,3000.00,3000.00,20859.70,28862.08,1045.61,1175.75,17277.75,20732.99
324,2021-11-21,187396.52,1060.70,3000.00,28862.08,1175.75,20732.99,54831.52,0.02,0.05,...,1051.16,907.91,3000.00,4365.16,28033.24,22287.97,1068.66,694.84,17021.54,23944.08
325,2021-11-22,103028.49,907.91,4365.16,22287.97,694.84,23944.08,52199.96,0.02,0.08,...,1060.70,1023.40,3000.00,2902.81,28862.08,18059.05,1175.75,0.00,20732.99,14781.64
326,2021-11-23,69067.49,1023.40,2902.81,18059.05,0.00,14781.64,36766.90,0.03,0.08,...,907.91,1001.10,4365.16,2900.00,22287.97,19983.83,694.84,0.00,23944.08,12440.56


In [76]:
sum = df.sum()
print(round(sum.bing_cost/sum.Revenue*100,2))
print(round(sum.google_ads_cost/sum.Revenue*100,2))
print(round(sum.facebook_cost/sum.Revenue*100,2))
print(round(sum.hulu_cost/sum.Revenue*100,2))
print(round(sum.pinterest_cost/sum.Revenue*100,2))

0.8
8.24
7.47
0.17
1.52


In [80]:
formula = 'Revenue ~ bing_cost + pinterest_cost + google_ads_cost + hulu_cost + facebook_cost + bing_cost_m1 + pinterest_cost_m1 + google_ads_cost_m1 + hulu_cost_m1 + facebook_cost_m1 + bing_cost_p1 + pinterest_cost_p1 + google_ads_cost_p1 + hulu_cost_p1 + facebook_cost_p1 + wkday'
lm = smf.formula.ols(formula=formula, data=df)
lm_results = lm.fit()
lm_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Revenue   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.636
Method:                 Least Squares   F-statistic:                     23.18
Date:                Mon, 29 Nov 2021   Prob (F-statistic):           9.96e-47
Time:                        00:50:16   Log-Likelihood:                -3176.6
No. Observations:                 268   AIC:                             6397.
Df Residuals:                     246   BIC:                             6476.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           6.909e+04   1.33e+04      5.210      0.000     4.3e+04    9.52e+04
wkday[T.mon]        8567.2015   1.05e+04      0.817      0.415   -1.21e+04    2.92e+04
wkday[T.sat]        8218.0721   8666.440      0.948      0.344   -8851.817    2.53e+04
wkday[T.sun]        1.581e+04   1.08e+04      1.468      0.143   -5402.910     3.7e+04
wkday[T.thu]       -1.563e+04   8542.947     -1.830      0.068   -3.25e+04    1193.971
wkday[T.tue]       -8108.1224   8994.615     -0.901      0.368   -2.58e+04    9608.158
wkday[T.wed]       -1.419e+04   8930.648     -1.589      0.113   -3.18e+04    3398.381
bing_cost             -1.5358      7.243     -0.212      0.832     -15.803      12.731
pinterest_cost        -1.4911     18.362     -0.081      0.935     -37.658      34.676
google_ads_cost        2.4673      1.387      1.778      0.077      -0.265       5.200
hulu_cost              0.9473      2.271      0.417      0.677      -3.526       5.421
facebook_cost         -3.5766      3.023     -1.183      0.238      -9.531       2.378
bing_cost_m1          13.4407      5.391      2.493      0.013       2.822      24.060
pinterest_cost_m1      6.7841     15.193      0.447      0.656     -23.142      36.710
google_ads_cost_m1    -0.4960      1.055     -0.470      0.639      -2.574       1.582
hulu_cost_m1           1.2165      2.274      0.535      0.593      -3.263       5.696
facebook_cost_m1       0.6893      2.121      0.325      0.746      -3.489       4.867
bing_cost_p1          11.3759      5.484      2.074      0.039       0.575      22.177
pinterest_cost_p1    -11.3753     12.286     -0.926      0.355     -35.575      12.824
google_ads_cost_p1     0.2437      0.978      0.249      0.803      -1.683       2.170
hulu_cost_p1          -1.8010      2.214     -0.813      0.417      -6.162       2.560
facebook_cost_p1       6.5045      2.065      3.151      0.002       2.438      10.571
==============================================================================
Omnibus:                      176.851   Durbin-Watson:                   1.124
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1986.016
Skew:                           2.505   Prob(JB):                         0.00
Kurtosis:                      15.360   Cond. No.                     2.97e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.97e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [46]:
formula = 'Revenue ~ bing_cost + pinterest_cost + google_ads_cost + hulu_cost + facebook_cost + bing_cost_m1 + pinterest_cost_m1 + google_ads_cost_m1 + hulu_cost_m1 + facebook_cost_m1'
lm = smf.formula.ols(formula=formula, data=df)
lm_results = lm.fit()
lm_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Revenue   R-squared:                       0.742
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     91.37
Date:                Sun, 28 Nov 2021   Prob (F-statistic):           3.64e-87
Time:                        19:53:06   Log-Likelihood:                -3899.3
No. Observations:                 328   AIC:                             7821.
Df Residuals:                     317   BIC:                             7862.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           2.359e+04   4750.495      4.965      0.000    1.42e+04    3.29e+04
bing_cost              8.3665      5.304      1.577      0.116      -2.069      18.802
pinterest_cost        -2.1605     14.140     -0.153      0.879     -29.981      25.660
google_ads_cost        3.1953      0.750      4.261      0.000       1.720       4.671
hulu_cost             -0.9532      2.186     -0.436      0.663      -5.254       3.347
facebook_cost          5.5050      1.989      2.768      0.006       1.592       9.418
bing_cost_m1           7.5214      5.266      1.428      0.154      -2.839      17.882
pinterest_cost_m1     24.0610     13.965      1.723      0.086      -3.416      51.538
google_ads_cost_m1    -0.8319      0.781     -1.065      0.288      -2.369       0.705
hulu_cost_m1           1.1189      2.217      0.505      0.614      -3.242       5.480
facebook_cost_m1      -3.3168      2.000     -1.658      0.098      -7.252       0.619
==============================================================================
Omnibus:                      212.120   Durbin-Watson:                   0.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2907.497
Skew:                           2.446   Prob(JB):                         0.00
Kurtosis:                      16.741   Cond. No.                     5.95e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.95e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [47]:
formula = 'Revenue ~ bing_cost + pinterest_cost + google_ads_cost + hulu_cost + facebook_cost + bing_cost_p1 + pinterest_cost_p1 + google_ads_cost_p1 + hulu_cost_p1 + facebook_cost_p1 + wkday'
lm = smf.formula.ols(formula=formula, data=df)
lm_results = lm.fit()
lm_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Revenue   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     67.00
Date:                Sun, 28 Nov 2021   Prob (F-statistic):           1.51e-90
Time:                        19:56:02   Log-Likelihood:                -3877.0
No. Observations:                 328   AIC:                             7788.
Df Residuals:                     311   BIC:                             7852.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           2.584e+04   6725.846      3.841      0.000    1.26e+04    3.91e+04
wkday[T.mon]        2714.9696   7670.468      0.354      0.724   -1.24e+04    1.78e+04
wkday[T.sat]        7310.9698   7116.356      1.027      0.305   -6691.323    2.13e+04
wkday[T.sun]        1.066e+04   8616.238      1.237      0.217   -6295.771    2.76e+04
wkday[T.thu]        -1.63e+04   7231.398     -2.254      0.025   -3.05e+04   -2068.907
wkday[T.tue]       -7758.9057   7375.454     -1.052      0.294   -2.23e+04    6753.193
wkday[T.wed]       -1.446e+04   7476.735     -1.934      0.054   -2.92e+04     253.060
bing_cost              8.3031      5.008      1.658      0.098      -1.551      18.158
pinterest_cost        20.0233     11.048      1.812      0.071      -1.715      41.761
google_ads_cost        2.2078      0.879      2.511      0.013       0.478       3.938
hulu_cost              0.8244      2.122      0.389      0.698      -3.350       4.999
facebook_cost         -3.6947      1.892     -1.952      0.052      -7.418       0.029
bing_cost_p1           5.8229      5.042      1.155      0.249      -4.097      15.743
pinterest_cost_p1     -1.1254     11.049     -0.102      0.919     -22.867      20.616
google_ads_cost_p1    -0.4294      0.858     -0.500      0.617      -2.118       1.260
hulu_cost_p1          -1.3708      2.081     -0.659      0.510      -5.465       2.723
facebook_cost_p1       7.3658      1.884      3.909      0.000       3.659      11.073
==============================================================================
Omnibus:                      234.612   Durbin-Watson:                   1.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3622.588
Skew:                           2.776   Prob(JB):                         0.00
Kurtosis:                      18.305   Cond. No.                     2.06e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
formula = 'Revenue ~ bing_cost + pinterest_cost + google_ads_cost + hulu_cost + facebook_cost + wkday'
lm = smf.formula.ols(formula=formula, data=df)
lm_results = lm.fit()
lm_results.summary(alpha=.2)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Revenue   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.619
Method:                 Least Squares   F-statistic:                     38.34
Date:                Mon, 29 Nov 2021   Prob (F-statistic):           6.31e-47
Time:                        20:02:26   Log-Likelihood:                -3021.5
No. Observations:                 254   AIC:                             6067.
Df Residuals:                     242   BIC:                             6109.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|        [0.1        0.9]
-----------------------------------------------------------------------------------
Intercept        9.512e+04   1.52e+04      6.259      0.000    7.56e+04    1.15e+05
wkday[T.mon]     -870.3071   8627.048     -0.101      0.920    -1.2e+04    1.02e+04
wkday[T.sat]     1105.6086   8688.178      0.127      0.899   -1.01e+04    1.23e+04
wkday[T.sun]     4392.7754   8928.226      0.492      0.623   -7080.528    1.59e+04
wkday[T.thu]    -1.959e+04   8586.306     -2.281      0.023   -3.06e+04   -8555.197
wkday[T.tue]    -1.079e+04   8514.758     -1.268      0.206   -2.17e+04     148.982
wkday[T.wed]    -2.607e+04   8516.278     -3.061      0.002    -3.7e+04   -1.51e+04
bing_cost          23.5508      3.510      6.710      0.000      19.040      28.061
pinterest_cost    -14.1423      8.836     -1.600      0.111     -25.498      -2.787
google_ads_cost     2.5323      0.511      4.957      0.000       1.876       3.189
hulu_cost          -0.3763      2.236     -0.168      0.866      -3.250       2.497
facebook_cost       3.3470      0.931      3.597      0.000       2.151       4.543
==============================================================================
Omnibus:                      163.275   Durbin-Watson:                   1.081
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1887.657
Skew:                           2.372   Prob(JB):                         0.00
Kurtosis:                      15.484   Cond. No.                     1.61e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.61e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
formula = 'Revenue ~ facebook_cost + pinterest_cost + google_ads_cost + bing_cost + hulu_cost + wkday'
lm = smf.formula.ols(formula=formula, data=df)
lm_results = lm.fit()
lm_results.summary(alpha=.2)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Revenue   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.619
Method:                 Least Squares   F-statistic:                     38.34
Date:                Mon, 29 Nov 2021   Prob (F-statistic):           6.31e-47
Time:                        22:30:24   Log-Likelihood:                -3021.5
No. Observations:                 254   AIC:                             6067.
Df Residuals:                     242   BIC:                             6109.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|        [0.1        0.9]
-----------------------------------------------------------------------------------
Intercept        9.512e+04   1.52e+04      6.259      0.000    7.56e+04    1.15e+05
wkday[T.mon]     -870.3071   8627.048     -0.101      0.920    -1.2e+04    1.02e+04
wkday[T.sat]     1105.6086   8688.178      0.127      0.899   -1.01e+04    1.23e+04
wkday[T.sun]     4392.7754   8928.226      0.492      0.623   -7080.528    1.59e+04
wkday[T.thu]    -1.959e+04   8586.306     -2.281      0.023   -3.06e+04   -8555.197
wkday[T.tue]    -1.079e+04   8514.758     -1.268      0.206   -2.17e+04     148.982
wkday[T.wed]    -2.607e+04   8516.278     -3.061      0.002    -3.7e+04   -1.51e+04
facebook_cost       3.3470      0.931      3.597      0.000       2.151       4.543
pinterest_cost    -14.1423      8.836     -1.600      0.111     -25.498      -2.787
google_ads_cost     2.5323      0.511      4.957      0.000       1.876       3.189
bing_cost          23.5508      3.510      6.710      0.000      19.040      28.061
hulu_cost          -0.3763      2.236     -0.168      0.866      -3.250       2.497
==============================================================================
Omnibus:                      163.275   Durbin-Watson:                   1.081
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1887.657
Skew:                           2.372   Prob(JB):                         0.00
Kurtosis:                      15.484   Cond. No.                     1.61e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.61e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""